# ✨ Jaras Email ✨
---

In [ ]:
import csv
import ssl
import smtplib
from email.message import EmailMessage
from datetime import datetime
from utils.email_utils import load_template
import logging
import os

# === CONFIGURATION ===
csv_path = "Input/testing_email.csv"
template_path = "templates/email_template.html"
subject = "Pemberian Sertifikat Schematics NPC 2024"


email_sender = "schematics2024@gmail.com"
email_password = "_"
log_file = "output/email_log.txt"
# ======================
 
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format="[{asctime}] {levelname} - {message}",
    style='{',
    datefmt="%Y-%m-%d %H:%M:%S"
)
template = load_template(template_path)

In [6]:
with open(csv_path, "r", encoding="utf-8") as data:
    reader = csv.DictReader(data)
    for i, row in enumerate(reader, start=1):
        if "email" not in row:
            logging.warning(f"Row {i} missing 'email' field. Skipped.")
            continue

        receiver_email = row["email"].strip()
        try:
            msg = EmailMessage()
            msg['From'] = email_sender
            msg['To'] = receiver_email
            msg['Subject'] = subject

            personalized_body = template.format(**row)
            msg.add_alternative(personalized_body, subtype='html')

            context = ssl.create_default_context()
            with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as smtp:
                smtp.login(email_sender, email_password)
                smtp.sendmail(email_sender, receiver_email, msg.as_string())

            logging.info(f"{i}. Email sent to {receiver_email}")
        except KeyError as ke:
            logging.error(f"{i}. Missing column in template: {ke}")
        except Exception as e:
            logging.error(f"{i}. Failed to send to {receiver_email}: {e}")